In [1]:
from tensorflow.keras.callbacks import Callback
import pandas as pd
import random 
import numpy as np
import math
from matplotlib import pyplot as plt
from matplotlib.pylab import mpl
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LeakyReLU,Conv1D,Dense
from tcn import TCN,tcn_full_summary
from sklearn.metrics import mean_squared_error # 均方误差
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import Input,Model,Sequential

### 导入模型

In [2]:
def rmse(y_pred, y_true):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [3]:
tcn = tf.keras.models.load_model('audio_4.h5',custom_objects={'TCN':TCN,'rmse':rmse})

In [4]:
path = 'H:/test_data/audio_data/test_audio.csv'
df = pd.read_csv(path,index_col=0)

In [5]:
X_Test=df.iloc[:,:-1]
Y_test=df.iloc[:,-1:]

### 对数据归一化

注意是（0，1）还是（-1，1）

In [6]:
df = df.iloc[:,:-1]
scaler = MinMaxScaler(feature_range=(-1,1)).fit(df)
X_test=pd.DataFrame(scaler.transform(X_Test))

### 转换为3维数据 [samples, timesteps, features]

In [7]:
x_test=X_test.values.reshape([X_test.shape[0],1,X_test.shape[1]])  
y_test=Y_test.values

In [8]:
x_test.shape,y_test.shape

((47, 1, 6373), (47, 1))

### 测试结果

In [9]:
score_test = tcn.evaluate(x_test,y_test)
score_test

2/2 [==============================] - 0s 2ms/step - loss: 6.3452 - mean_absolute_error: 5.3443


[6.345202922821045, 5.34426736831665]

In [10]:
real_predict = tcn.predict(x_test)
real_y = y_test
real_y = real_y.reshape(len(real_y),1)
real_predict = real_predict.reshape(len(real_predict),1)

In [11]:
sum1 = 0
print(real_y.shape[0])
for i in range(real_y.shape[0]):
#     print(real_y[i],real_predict[i])
    if(real_y[i]>=10 and real_predict[i]>=10):
        sum1 = sum1 + 1
    if(real_y[i]<10 and real_predict[i]<10):
        sum1 = sum1 + 1
    if(real_y[i]<10 and real_predict[i]>10):
        print(real_y[i],real_predict[i])
    if(real_y[i]>10 and real_predict[i]<10):
        print(real_y[i],real_predict[i])
print(sum1)

47
[15] [2.0869107]
[1] [10.1331]
[18] [8.845518]
[15] [6.1318502]
[17] [8.063643]
[12] [2.9947793]
[3] [12.125185]
[17] [5.201592]
[17] [6.123598]
[2] [10.0925045]
[3] [12.358919]
34
